In [1]:
import numpy as np
import os
import pickle

from keras.utils import np_utils
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, concatenate
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta
from matplotlib import pyplot as plt 
import string

Using TensorFlow backend.
/home/wei/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### 配置参数

In [2]:
captcha_word  = '0123456789+-* '

width = 350
height = 80

word_len = 8
word_class = len(captcha_word)

train_dir = '/home/wei/data/first/first_train'
train_label =  '/home/wei/data/first/first_label.txt'

#生成字符索引
char_indices = dict((c, i) for i,c in enumerate(captcha_word))
indices_char = dict((i, c) for i,c in enumerate(captcha_word))


#验证码字符串转数组
def captcha_to_vec(captcha):    
   
    vector = np.zeros(word_len * word_class)
    for i,ch in enumerate(captcha):
        if ch == ',':
            continue
        idex = i * word_class + char_indices[ch]
        vector[idex] = 1
    return vector

#把数组转换回文字
def vec_to_captcha(vec):
    text = []
    vec[vec < 0.5] = 0    
    char_pos = vec.nonzero()[0]
    for i, ch in enumerate(char_pos):
        text.append(captcha_word[ch % word_class])
    return ''.join(text)


In [3]:

image_list = []

for item in os.listdir(train_dir):
    image_list.append(item)

print("ok")

ok


In [4]:

X = np.zeros((len(image_list), height, width, 3), dtype = np.uint8)
y = np.zeros((len(image_list), word_len * word_class), dtype = np.uint8)

with open(train_label) as file_object:
    p = file_object.readlines()
for i,img in enumerate(image_list):
    img_path = train_dir + "/" + img
    raw_img = image.load_img(img_path, target_size=(height, width))
    X[i] = image.img_to_array(raw_img)
    for j in range(len(p)):
        if img[:4]==p[j][:4]:
            break;
    s = ''
    flage = 0
    for k in p[j]:
        if k=='=':
            break
        else:
            s += k
    n = len(s[5:])
    if n!= 8:
        for m in range(8-n):
            s += ' '
    y[i] = captcha_to_vec(s[5:])

print("ok")


ok


In [5]:
#保存成pkl文件
file = open('/home/wei/data/first/val/first_train_data.pkl','wb')
pickle.dump((X,y) , file)
print("ok")

ok


### 加载数据，读取pickle文件

In [3]:
#读取pickle文件
file = open('/home/wei/data/first/val/first_train_data.pkl', 'rb')
X, y = pickle.load(file)
print("ok")

ok


### 创建模型

In [4]:

input_tensor = Input( shape=(height, width, 3))

x = input_tensor


x = Convolution2D(32, 3, padding='same', activation='relu')(x)
x = Convolution2D(32, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)


x = Convolution2D(64, 3, padding='same', activation='relu')(x)
x = Convolution2D(64, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Convolution2D(128, 3, padding='same', activation='relu')(x)
x = Convolution2D(128, 3, padding='same',activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.25)(x)


#最后连接5个分类器
x = [Dense(word_class, activation='softmax', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)

model = Model(inputs=input_tensor, outputs=output)

opt = Adadelta(lr=0.1)
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="/home/wei/weight/weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", 
                               verbose=2, save_weights_only=True)

print("ok")


ok


### 开始训练模型

In [9]:
model.fit(X, y, epochs=55,callbacks=[checkpointer],validation_split=0.1)

Train on 8100 samples, validate on 900 samples
Epoch 1/55
8096/8100 [============================>.] - ETA: 0s - loss: 100.1600 - acc: 0.0485
Epoch 00001: saving model to /home/wei/weight/weights.01--100.04-0.0544.hdf5
8100/8100 [==============================] - 1350s 167ms/step - loss: 100.1552 - acc: 0.0486 - val_loss: 100.0407 - val_acc: 0.0544
Epoch 2/55
8096/8100 [============================>.] - ETA: 0s - loss: 99.9579 - acc: 0.0495
Epoch 00002: saving model to /home/wei/weight/weights.02--100.04-0.0544.hdf5
8100/8100 [==============================] - 1329s 164ms/step - loss: 99.9549 - acc: 0.0495 - val_loss: 100.0407 - val_acc: 0.0544
Epoch 3/55
8096/8100 [============================>.] - ETA: 0s - loss: 99.9658 - acc: 0.0493
Epoch 00003: saving model to /home/wei/weight/weights.03--100.04-0.0544.hdf5
8100/8100 [==============================] - 1330s 164ms/step - loss: 99.9697 - acc: 0.0493 - val_loss: 100.0407 - val_acc: 0.0544
Epoch 4/55
8096/8100 [=======================

8096/8100 [============================>.] - ETA: 0s - loss: 16.6366 - acc: 0.9928
Epoch 00028: saving model to /home/wei/weight/weights.28--16.64-1.0000.hdf5
8100/8100 [==============================] - 1330s 164ms/step - loss: 16.6366 - acc: 0.9927 - val_loss: 16.6355 - val_acc: 1.0000
Epoch 29/55
8096/8100 [============================>.] - ETA: 0s - loss: 16.6357 - acc: 0.9889
Epoch 00029: saving model to /home/wei/weight/weights.29--16.64-1.0000.hdf5
8100/8100 [==============================] - 1330s 164ms/step - loss: 16.6357 - acc: 0.9889 - val_loss: 16.6355 - val_acc: 1.0000
Epoch 30/55
8096/8100 [============================>.] - ETA: 0s - loss: 16.6356 - acc: 0.9942
Epoch 00030: saving model to /home/wei/weight/weights.30--16.64-1.0000.hdf5
8100/8100 [==============================] - 1330s 164ms/step - loss: 16.6356 - acc: 0.9942 - val_loss: 16.6355 - val_acc: 1.0000
Epoch 31/55
8096/8100 [============================>.] - ETA: 0s - loss: 16.6355 - acc: 0.9956
Epoch 00031: s

In [12]:
#保存权重和模型
model.save_weights('/home/wei/data/first/val/first_model_weights.h5')
model.save('/home/wei/data/first/val/first_model.h5')